<i>Authors:</i><br>
<i>- Jun Ki Min (https://github.com/loomlike)</i><br>
<i>- JS Tan (https://github.com/jiata)</i>



# Setup Multiple Virtual Machines with Multiple Users

This notebook shows how to setup multiple Data Science Virtual Machines (DSVMs) with multiple user accounts.
More specifically, it deploys Azure VMSS (Virtual Machine Scale Set) and invokes the post-deployment-script on each VM instance
to:
1. Clone an example project repository,
1. Setup conda environment for the project, and
1. Create multiple JupyterHub users (Each user can access JupyterHub by opening `https://vm-ip-address:8000`)

## Requirements
* Azure CLI (https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest)
* Python v3.5 or greater

## Azure CLI Setup

In [ ]:
!az login

Set following parameters

In [ ]:
AZURE_SUBSCRIPTION_ID = "your-subsription-id"
LOCATION = "location-to-deploy"  # e.g. australiaeast
VM_SIZE = "your-dsvm-size"  # e.g. Standard_NC6s_v3
RG_NAME = "your-resource-group-name"
ADMIN_ID = "your-admin-id-to-use"
ADMIN_PW = "your-admin-password-to-use"

# Number of VM instances in a scale set
NUM_VM = 2
NUM_USERS = 2

In [ ]:
!az account set -s {AZURE_SUBSCRIPTION_ID}

In [ ]:
!az account show

## Create VMSS

In [ ]:
VMSS_NAME = "{}-vmss".format(RG_NAME)

In [ ]:
!az group create --name {RG_NAME} --location {LOCATION}

In [ ]:
# Setup public-ip for each vm by using `--public-ip-per-vm` parameter
!az vmss create -g {RG_NAME} -n {VMSS_NAME} --instance-count {NUM_VM} --image microsoft-dsvm:linux-data-science-vm-ubuntu:linuxdsvmubuntu:latest --vm-sku {VM_SIZE} --public-ip-per-vm --admin-username {ADMIN_ID} --admin-password {ADMIN_PW}

In [ ]:
# Check public ip addresses of the instances
!az vmss list-instance-public-ips --resource-group {RG_NAME} --name {VMSS_NAME} | grep 'ipAddress'

## Create User Accounts and Environment Setup

Edit following scripts to fit your use-case. In this example, we:
1. Clone the microsoft/computervision repository,
1. Create a conda environment and its jupyter kernel,
1. Create jupyterhub users and copy the repository to each user's local folder

In [ ]:
SCRIPT = "vm_user_env_setup.sh"

users = "\", \"".join(["user{}".format(u) for u in range(1, NUM_USERS+1)])
setup_sh = """#!/bin/bash
cd /data

# clone repo and install the conda env 
git clone https://www.github.com/microsoft/computervision 
# change permission as we copy this into each user's folder
chmod -R ugo+rwx /data/computervision

# enable conda if not done yet
. /data/anaconda/etc/profile.d/conda.sh

# create conda env and kernel
conda env create -f /data/computervision/environment.yml --name cv
conda activate cv 
python -m ipykernel install --name cv --display-name "MLADS CV LAB" 

# add users to jupyterhub
echo 'c.Authenticator.whitelist = {{"{users}"}}' | tee -a /etc/jupyterhub/jupyterhub_config.py

# create the users on the vm 
for i in $(seq {num_users})
do
    USERNAME=user$i
    PASSWORD=password$i
    adduser --quiet --disabled-password --gecos "" $USERNAME
    echo "$USERNAME:$PASSWORD" | chpasswd
    rm -rf /data/home/$USERNAME/notebooks/*
    # copy repo
    cp -ar /data/computervision /data/home/$USERNAME/notebooks
done

# restart jupyterhub service
systemctl stop jupyterhub 
systemctl start jupyterhub 

exit
""".format(
    admin_id=ADMIN_ID,
    users=users,
    num_users=NUM_USERS,
)

with open(SCRIPT, "w") as f:
    f.write(setup_sh)

In [ ]:
!az vmss list-instances -n {VMSS_NAME} -g {RG_NAME} --query "[].id" --output tsv | az vmss run-command invoke --command-id RunShellScript --scripts @{SCRIPT} --ids @- 